In [29]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video
from elastica.modules import BaseSystemCollection, Constraints, Forcing, Damping 

from elastica.rod.cosserat_rod import CosseratRod 
from elastica.dissipation import AnalyticalLinearDamper
from elastica.boundary_conditions import OneEndFixedBC
from elastica.external_forces import EndpointForces, GravityForces 
from elastica import Connections
from elastica import FixedJoint
from elastica.callback_functions import CallBackBaseClass
from elastica.timestepper import integrate, PositionVerlet
from elastica import CallBacks

from elastica.timestepper.symplectic_steppers import PositionVerlet
from collections import defaultdict
from magneto_pyelastica import *

In [150]:
#--------simulator----------
class UnitMagneticRodSimulator(BaseSystemCollection, Constraints, Forcing, Damping, Connections,CallBacks):
    pass
Uniform_M_Sim = UnitMagneticRodSimulator()
#--------mmGS unit----------
density = 2.273  #mg/mm^3 
base_length = 3 #mm 
base_radius = 0.3 #mm 
scale_E = 1e-3 #scale dowing Young's modulus
E = 1.4e9 * scale_E #Young's modulus
shear_modulus = E/3 * scale_E # shear modulus


dt = 1e-4 # time step
nu = 5
endtime = 5

#--------rod definition-------
n_elem = 20
start = np.array([0.0,0.0,0.0])
direction = np.array([0.0,1.0,0.0])
normal = np.array([1.0,0.0,0.0])
M_rod = CosseratRod.straight_rod(n_elem, start, direction, normal, base_length, base_radius, density, youngs_modulus=E, shear_modulus=shear_modulus)

Uniform_M_Sim.append(M_rod)
#--------magnetic properties-----
magnetization_density = 1.28e5 * 1e-3 #A/mm
magnetic_field_angle = np.pi/2
magnetic_field = 90e-3*1e6*scale_E # T, mg/(s^2*A)
#------test----------
# magnetization_density = 1e5
# magnetic_field_angle = np.pi/2
# magnetic_field = 10e-2

magnetization_direction = np.ones((n_elem)) * direction.reshape(3, 1)

#------set the constant magnetic field object-----
magnetic_field_amplitude = magnetic_field* np.array([0.0, np.cos(magnetic_field_angle), np.sin(magnetic_field_angle)])
magnetic_field_object = ConstantMagneticField(
    magnetic_field_amplitude, ramp_interval = 0.5, start_time = 0.5, end_time = endtime
)

#--------constrain----------
Uniform_M_Sim.constrain(M_rod).using(
    OneEndFixedBC, constrained_position_idx=(0,), constrained_director_idx=(0,)
)

#--------damping------------
Uniform_M_Sim.dampen(M_rod).using(
    AnalyticalLinearDamper, damping_constant = nu, time_step = dt
)
#--------force--------------
Uniform_M_Sim.add_forcing_to(M_rod).using(
    MagneticForces,
    external_magnetic_field = magnetic_field_object,
    magnetization_density = magnetization_density,
    magnetization_direction = magnetization_direction,
    rod_volume = M_rod.volume,
    rod_director_collection = M_rod.director_collection.copy(),
)
#------callback function------
class MagneticRodCallBack(CallBackBaseClass):
    def __init__(self, step_skip:int, callback_params:dict):
        super().__init__()
        self.step_skip = step_skip
        self.callback_params = callback_params
    
    def make_callback(self, system, time, current_step: int):
        if current_step % self.step_skip == 0:
            self.callback_params["time"].append(time)
            self.callback_params["position"].append(system.position_collection.copy())
            self.callback_params["velocity"].append(system.velocity_collection.copy())
            return
MR_list = defaultdict(list)
Uniform_M_Sim.collect_diagnostics(M_rod).using(
    MagneticRodCallBack, step_skip = 100, callback_params = MR_list
)

Uniform_M_Sim.finalize()
#--------time integration-----
timestepper = PositionVerlet()
final_time =endtime
total_steps = int(final_time / dt)
integrate(timestepper, Uniform_M_Sim, final_time, total_steps)


100%|██████████| 50000/50000 [00:06<00:00, 7967.81it/s]

Final time of simulation is :  4.999999999995016


4.999999999995016

In [153]:
import os
from IPython.display import Video
from Plot_Method import plot_video_2D
current_dir = os.path.abspath("")
filename = current_dir + '/'+"Uniform_Magnetic_Rod.mp4"
x_lim = np.array([-1, 6.1])
y_lim = np.array([-3,3])
plot_video_2D(normal, x_lim, y_lim, MR_list, video_name=filename, fps=10)
Video(filename)


Creating 2D video -- this can take a few minutes--------------------
